# **K-Means Clustering with PCA and Silhouette Analysis**
# **CMPT 459 Course Project**

This notebook demonstrates **K-Means clustering** on the diabetic patient dataset.

## What we'll do:
* Data preprocessing (consistent with project pipeline)
* **PCA (50 components)** for dimensionality reduction
* **Silhouette Score analysis** to find optimal k
* 2D visualization of clusters
* Comparison with true readmission labels

**References**: `cluster_analysis.py`, `kmeans.py`


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from kmeans import KMeans

## **1. Data Loading & Preprocessing**

Standard preprocessing:
* Replace `'?'` with NaN
* Drop columns with >40% missing
* Encode categorical variables
* Normalize numerical features
* Remove ID columns


In [2]:
def load_and_preprocess_data(data_path):
    """Load and preprocess the diabetic patient data."""
    print("Loading data...")
    df = pd.read_csv(data_path)
    print(f"Original shape: {df.shape}")
    
    df = df.replace("?", np.nan)
    threshold = 0.4 * len(df)
    df = df.dropna(thresh=threshold, axis=1)
    
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].fillna("Unknown")
    
    df["readmitted"] = df["readmitted"].map({"NO": 0, ">30": 1, "<30": 2})
    
    cat_cols = df.select_dtypes(include="object").columns
    le = LabelEncoder()
    
    for col in cat_cols:
        if df[col].nunique() < 10:
            df[col] = le.fit_transform(df[col].astype(str))
        else:
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    
    target = df["readmitted"].copy() if "readmitted" in df.columns else None
    features_df = df.drop(columns=["readmitted"]) if "readmitted" in df.columns else df
    
    for col in ["encounter_id", "patient_nbr"]:
        if col in features_df.columns:
            features_df = features_df.drop(columns=[col])
    
    num_cols = features_df.select_dtypes(include=["int64", "float64"]).columns
    scaler = StandardScaler()
    features_df[num_cols] = scaler.fit_transform(features_df[num_cols])
    
    X = features_df.values
    print(f"Final shape: {X.shape}")
    return X, target

X, target = load_and_preprocess_data("data/diabetic_data.csv")

Loading data...
Original shape: (101766, 50)
Final shape: (101766, 2389)


## **2. PCA Dimensionality Reduction**

Reduce to 50 components (preserves ~85-90% variance):
* Reduces noise
* Speeds up clustering
* Mitigates curse of dimensionality


In [3]:
pca_model = PCA(n_components=50, random_state=42)
X_pca = pca_model.fit_transform(X)
explained_var = np.sum(pca_model.explained_variance_ratio_)

print(f"PCA shape: {X_pca.shape}")
print(f"Explained variance: {explained_var:.4f} ({explained_var*100:.2f}%)")

PCA shape: (101766, 50)
Explained variance: 0.9271 (92.71%)


## **3. K-Means with Silhouette Analysis**

Test k=2 to k=6 and evaluate using **Silhouette Score**:
* Score ranges from -1 to 1
* Higher = better-defined clusters
* We use **K-Means++** initialization


In [ ]:
ks = list(range(2, 7))
silhouettes = []
best_k = None
best_score = -1
best_clustering = None

print("=" * 70)
print("K-MEANS CLUSTERING")
print("=" * 70)

for k in ks:
    print(f"\nTesting k={k}...", end=" ")
    kmeans = KMeans(n_clusters=k, init="kmeans++")
    clustering = kmeans.fit(X_pca)
    sil_score = kmeans.silhouette(clustering, X_pca)
    silhouettes.append(sil_score)
    print(f"Silhouette: {sil_score:.4f}")
    
    if sil_score > best_score:
        best_score = sil_score
        best_k = k
        best_clustering = clustering.copy()

print(f"\n\nBest k={best_k} (Silhouette={best_score:.4f})")

## **4. Visualization**
### 4.1 Silhouette Score Plot


**Saved Result:**

<img src="../kmeans_results/silhouette_scores.png" alt="Silhouette Scores" width="800"/>


### 4.2 Cluster Visualization (2D PCA)


**Saved Result:**

<img src="../kmeans_results/clustering_kmeanspp_k2.png" alt="K-Means Clustering (K-Means++ Initialization)" width="800"/>


**Additional Visualizations:**

<img src="../kmeans_results/silhouette_comparison.png" alt="Silhouette Comparison" width="800"/>

**Different Initialization Methods:**

<img src="../kmeans_results/clustering_random_k2.png" alt="Random Initialization" width="800"/>

<img src="../kmeans_results/clustering_k2.png" alt="Standard K-Means (k=2)" width="800"/>


### 4.3 Comparison with True Readmission Labels


In [ ]:
if target is not None:
    plt.figure(figsize=(10, 8))
    colors = ["red", "orange", "green"]
    labels = ["No Readmission", ">30 days", "<30 days"]
    
    for i, val in enumerate(np.unique(target)):
        mask = (target == val)
        plt.scatter(X_vis[mask, 0], X_vis[mask, 1],
                   color=colors[i], label=labels[i], alpha=0.6, s=20)
    
    plt.xlabel("PC1", fontsize=12)
    plt.ylabel("PC2", fontsize=12)
    plt.title("True Readmission Labels in PCA Space", fontsize=14, fontweight="bold")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

## **Interpretation & Discussion**

### **Key Findings**

1. **Optimal k**: The silhouette analysis typically shows k=2 as optimal
   - Suggests natural binary split in patient data
   - Could represent high-risk vs low-risk groups

2. **Cluster Quality**: Moderate silhouette scores (0.2-0.4) indicate:
   - Some overlap between clusters
   - Continuous variation rather than distinct groups
   - High dimensionality makes perfect separation difficult

3. **Relationship to Readmission**:
   - K-Means is unsupervised (doesn't use labels)
   - Discovered clusters don't directly match readmission categories
   - However, they may capture underlying risk factors

### **Strengths**
* Simple and interpretable
* Scalable to large datasets
* Fast with K-Means++ initialization
* Useful for patient segmentation

### **Limitations**
* Assumes spherical clusters
* Sensitive to initialization (despite K-Means++)
* Requires choosing k
* Doesn't handle outliers well

### **Practical Applications**
1. Patient stratification (high-risk vs low-risk)
2. Resource allocation
3. Feature engineering (cluster membership as a feature)
4. Hypothesis generation for clinical research
